# A day with *(the)* Julia *(language)*  

## Análisis de datos   
   

In [1]:
ENV["LINES"]   = 10 
ENV["COLUMNS"] = 60;

- *DataFrame* para manejar datos tabulares, datos faltantes...
- *Gadfly* para hacer rápidamente gráficos estadísticos al estilo de *ggplot2* 
- *Plots* para graficar independiente del *backend*, soporte para animaciones, gráficos interactivos con *Plotly* o 3D, etc.
- *IJulia* para trabajar con IPython/**Ju**pyter notebooks, e *Interact*
- Test estadísticos, clustering, etc. con los paquetes de la organización [**JuliaStats**](http://juliastats.github.io/)

In [2]:
using DataFrames
benchmark = readtable(joinpath("data","benchmark.csv"), separator=';')

,Benchmark,Fortran,Julia,Python,R,Matlab,Octave,Mathematica,JavaScript,Go,LuaJIT,Java
1,fib,0.7,2.11,77.76,533.52,26.89,9324.35,118.53,3.36,1.86,1.71,1.21
2,parse_int,5.05,1.45,17.02,45.73,802.52,9581.44,15.02,6.06,1.2,5.77,3.35
3,quicksort,1.31,1.15,32.89,264.54,4.92,1866.01,43.23,2.7,1.29,2.03,2.6
4,mandel,0.81,0.79,15.32,53.16,7.58,451.81,5.13,0.66,1.11,0.67,1.35
5,pi_sum,1.0,1.0,21.99,9.56,1.0,299.31,1.69,1.01,1.0,1.0,1.0
6,rand_mat_stat,1.45,1.66,17.93,14.56,14.52,30.93,5.95,2.3,2.96,3.27,3.92
7,rand_mat_mul,3.48,1.02,1.14,1.57,1.12,1.12,1.3,15.07,1.42,1.16,2.36


In [3]:
longformat = melt(benchmark, [:Benchmark]) # Reshaping: wide -> long

,variable,value,Benchmark
1,Fortran,0.7,fib
2,Fortran,5.05,parse_int
3,Fortran,1.31,quicksort
4,Fortran,0.81,mandel
5,Fortran,1.0,pi_sum
6,Fortran,1.45,rand_mat_stat
7,Fortran,3.48,rand_mat_mul
8,Julia,2.11,fib
9,Julia,1.45,parse_int
10,Julia,1.15,quicksort


In [4]:
meantime = by(longformat, :variable, df -> mean( df[:value] )) # Split-Apply-Combine

,variable,x1
1,Fortran,1.9714285714285715
2,Go,1.5485714285714285
3,Java,2.255714285714286
4,JavaScript,4.451428571428571
5,Julia,1.3114285714285712
6,LuaJIT,2.23
7,Mathematica,27.264285714285712
8,Matlab,122.64999999999999
9,Octave,3079.281428571429
10,Python,26.292857142857144


In [5]:
sort!(meantime, cols=[:x1]) # Sorting
names!(meantime, [:Lenguaje, :Tiempo_Medio])

,Lenguaje,Tiempo_Medio
1,Julia,1.3114285714285712
2,Go,1.5485714285714285
3,Fortran,1.9714285714285715
4,LuaJIT,2.23
5,Java,2.255714285714286
6,JavaScript,4.451428571428571
7,Python,26.292857142857144
8,Mathematica,27.264285714285712
9,Matlab,122.64999999999999
10,R,131.80571428571426


Descripción (estadística) del dataset (columnas), similar a `summary` de *R*.

In [7]:
describe(meantime)

Lenguaje
Length  11
Type    Symbol
NAs     0
NA%     0.0%
Unique  11

Tiempo_Medio
Min      1.3114285714285712
1st Qu.  2.100714285714286
Median   4.451428571428571
Mean     309.1875324675325
3rd Qu.  74.95714285714286
Max      3079.281428571429
NAs      0
NA%      0.0%



### Usando funciones de R

In [8]:
using RCall

In [9]:
ks = R"""ks.test($(benchmark[:Julia]), $(benchmark[:R]))"""

RCall.RObject{RCall.VecSxp}

	Two-sample Kolmogorov-Smirnov test

data:  $(benchmark[:Julia]) and $(benchmark[:R])
D = 0.85714, p-value = 0.008159
alternative hypothesis: two-sided



In [10]:
rcopy(ks) # Transforma los datos de R a tipos de datos de Julia

Dict{Symbol,Any} with 5 entries:
  :statistic         => 0.8571428571428571
  :alternative       => "two-sided"
  symbol("p.value")  => 0.008158508158508049
  :method            => "Two-sample Kolmogorov-Smirnov test"
  symbol("data.name… => "\$(benchmark[:Julia]) and \$(bench…

### JuliaStats

Muchos de los tests estadísticos más clásicos están definidos en alguna biblioteca de Julia

In [11]:
using HypothesisTests

In [12]:
ApproximateTwoSampleKSTest(benchmark[:Julia], benchmark[:R])

Approximate two sample Kolmogorov-Smirnov test
----------------------------------------------
Population details:
    parameter of interest:   Supremum of CDF differences
    value under h_0:         0.0
    point estimate:          0.857142857142857

Test summary:
    outcome with 95% confidence: reject h_0
    two-sided p-value:           0.011681952358448919 (significant)

Details:
    number of observations:   [7,7]
    KS-statistic:              1.603567451474546


In [13]:
SignedRankTest(
convert(Vector{Float64}, benchmark[:Julia]), 
convert(Vector{Float64}, benchmark[:R])
) # Wilcoxon signed rank test

Exact Wilcoxon signed rank test
-------------------------------
Population details:
    parameter of interest:   Location parameter (pseudomedian)
    value under h_0:         0
    point estimate:          -44.279999999999994

Test summary:
    outcome with 95% confidence: reject h_0
    two-sided p-value:           0.015625000000000003 (significant)

Details:
    number of observations:      7
    Wilcoxon rank-sum statistic: 0.0
    rank sums:                   [0.0,28.0]
    adjustment for ties:         0.0


In [14]:
using GLM

lineal = lm(R ~ Julia, benchmark)

DataFrames.DataFrameRegressionModel{GLM.LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredQR{Float64}},Float64}

Formula: R ~ 1 + Julia

Coefficients:
             Estimate Std.Error t value Pr(>|t|)
(Intercept)  -247.159   200.519 -1.2326   0.2725
Julia         288.971   145.487 1.98623   0.1037


In [15]:
inter, pend = coef(lineal)

2-element Array{Float64,1}:
 -247.159
  288.971

In [20]:
using Plots

In [16]:
scatter(benchmark[:Julia], benchmark[:R], legend=false, xlab="Julia", ylab="R")
abline!(pend, inter)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Created with matplotlib (http://matplotlib.org/) -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 0.6 -->
 
 <path d="
M4.15625 35.2969
Q4.15625 48 6.76562 55.7344
Q9.375 63.4844 14.5156 67.6719
Q19.6719 71.875 27.4844 71.875
Q33.25 71.875 37.5938 69.5469
Q41.9375 67.2344 44.7656 62.8594
Q47.6094 58.5 49.2188 52.2188
Q50.8281 45.9531 50.8281 35.2969
Q50.8281 22.7031 48.2344 14.9688
Q45.6562 7.23438 40.5 3
Q35.3594 -1.21875 27.4844 -1.21875
Q17.1406 -1.21875 11.2344 6.20312
Q4.15625 15.1406 4.15625 35.2969
M13.1875 35.2969
Q13.1875 17.6719 17.3125 11.8281
Q21.4375 6 27.4844 6
Q33.5469 6 37.6719 11.8594
Q41.7969 17.7188 41.7969 35.2969
Q41.7969 52.9844 37.6719 58.7812
Q33.5469 64.5938 27.3906 64.5938
Q21.3438 64.5938 17.7188 59.4688
Q13.1875 52.9375 13.1875 35.2969" id="ArialMT-30"/>
 
 <path d="
M49.75 54.0469
L41.0156 53.375
Q39.8438 58.5469 37.7031 60.8906
Q34.125 64.6562 28.9062 64.6562
Q24.7031 64.6562 21.5312 62.3125
Q17.3906 59.2812 14.9844 53.4688
Q12.5938 47.6562 12.5 36.9219
Q15.6719 41.75 20.2656 44.0938
Q24.8594 46.4375 29.8906 46.4375
Q38.6719 46.4375 44.8438 39.9688
Q51.0312 33.5 51.0312 23.25
Q51.0312 16.5 48.125 10.7188
Q45.2188 4.9375 40.1406 1.85938
Q35.0625 -1.21875 28.6094 -1.21875
Q17.625 -1.21875 10.6875 6.85938
Q3.76562 14.9375 3.76562 33.5
Q3.76562 54.25 11.4219 63.6719
Q18.1094 71.875 29.4375 71.875
Q37.8906 71.875 43.2812 67.1406
Q48.6875 62.4062 49.75 54.0469
M13.875 23.1875
Q13.875 18.6562 15.7969 14.5
Q17.7188 10.3594 21.1875 8.17188
Q24.6562 6 28.4688 6
Q34.0312 6 38.0312 10.4844
Q42.0469 14.9844 42.0469 22.7031
Q42.0469 30.125 38.0781 34.3906
Q34.125 38.6719 28.125 38.6719
Q22.1719 38.6719 18.0156 34.3906
Q13.875 30.125 13.875 23.1875" id="ArialMT-36"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 0.8 -->
 
 <path d="
M17.6719 38.8125
Q12.2031 40.8281 9.5625 44.5312
Q6.9375 48.25 6.9375 53.4219
Q6.9375 61.2344 12.5469 66.5469
Q18.1719 71.875 27.4844 71.875
Q36.8594 71.875 42.5781 66.4219
Q48.2969 60.9844 48.2969 53.1719
Q48.2969 48.1875 45.6719 44.5
Q43.0625 40.8281 37.75 38.8125
Q44.3438 36.6719 47.7812 31.875
Q51.2188 27.0938 51.2188 20.4531
Q51.2188 11.2812 44.7188 5.03125
Q38.2344 -1.21875 27.6406 -1.21875
Q17.0469 -1.21875 10.5469 5.04688
Q4.04688 11.3281 4.04688 20.7031
Q4.04688 27.6875 7.59375 32.3906
Q11.1406 37.1094 17.6719 38.8125
M15.9219 53.7188
Q15.9219 48.6406 19.1875 45.4062
Q22.4688 42.1875 27.6875 42.1875
Q32.7656 42.1875 36.0156 45.375
Q39.2656 48.5781 39.2656 53.2188
Q39.2656 58.0625 35.9062 61.3594
Q32.5625 64.6562 27.5938 64.6562
Q22.5625 64.6562 19.2344 61.4219
Q15.9219 58.2031 15.9219 53.7188
M13.0938 20.6562
Q13.0938 16.8906 14.875 13.375
Q16.6562 9.85938 20.1719 7.92188
Q23.6875 6 27.7344 6
Q34.0312 6 38.125 10.0469
Q42.2344 14.1094 42.2344 20.3594
Q42.2344 26.7031 38.0156 30.8594
Q33.7969 35.0156 27.4375 35.0156
Q21.2344 35.0156 17.1562 30.9062
Q13.0938 26.8125 13.0938 20.6562" id="ArialMT-38"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 1.0 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 1.2 -->
 
 <path d="
M50.3438 8.45312
L50.3438 0
L3.03125 0
Q2.9375 3.17188 4.04688 6.10938
Q5.85938 10.9375 9.82812 15.625
Q13.8125 20.3125 21.3438 26.4688
Q33.0156 36.0312 37.1094 41.625
Q41.2188 47.2188 41.2188 52.2031
Q41.2188 57.4219 37.4688 61
Q33.7344 64.5938 27.7344 64.5938
Q21.3906 64.5938 17.5781 60.7812
Q13.7656 56.9844 13.7188 50.25
L4.6875 51.1719
Q5.60938 61.2812 11.6562 66.5781
Q17.7188 71.875 27.9375 71.875
Q38.2344 71.875 44.2344 66.1562
Q50.25 60.4531 50.25 52
Q50.25 47.7031 48.4844 43.5469
Q46.7344 39.4062 42.6562 34.8125
Q38.5781 30.2188 29.1094 22.2188
Q21.1875 15.5781 18.9375 13.2031
Q16.7031 10.8438 15.2344 8.45312
z
" id="ArialMT-32"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 1.4 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 1.

In [17]:
using Distributions

β = rand(Beta(2,2), 1000)

1000-element Array{Float64,1}:
 0.25109  
 0.487044 
 0.771439 
 ⋮        
 0.0467534
 0.403369 

### Usando bibliotecas de R

In [18]:
# R"""
# install.packages("modeest")
# """

@rimport modeest as rmode

β_mode = rmode.hsm(β) # half-sample mode

This is package 'modeest' written by P. PONCET.
For a complete list of functions, use 'library(help = "modeest")' or 'help.start()'.



RCall.RObject{RCall.RealSxp}
[1] 0.4734172


In [19]:
rcopy(β_mode)

0.4734171729333303

## Clustering

In [31]:
iris = readtable(joinpath("data", "iris.csv"))

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa
7,4.6,3.4,1.4,0.3,setosa
8,5.0,3.4,1.5,0.2,setosa
9,4.4,2.9,1.4,0.2,setosa
10,4.9,3.1,1.5,0.1,setosa


In [37]:
using Clustering

cl = kmeans(convert(Matrix{Float64}, iris[:, [:PetalWidth, :PetalLength]])', 3)

Clustering.KmeansResult{Float64}(2x3 Array{Float64,2}:
 1.35926  0.246  2.04783
 4.29259  1.462  5.62609,[2,2,2,2,2,2,2,2,2,2  …  3,3,3,3,3,3,3,3,3,3],[0.00596,0.00596,0.02836,0.00356,0.00596,0.08036,0.00676,0.00356,0.00596,0.02276  …  0.124707,0.340359,0.29862,0.13862,0.209924,0.245142,0.413837,0.183837,0.114707,0.338185],[54,50,46],[54.0,50.0,46.0],31.412885668276868,9,true)

In [35]:
cl.centers'

3x2 Array{Float64,2}:
 2.0375   5.59583
 0.246    1.462  
 1.34231  4.26923

In [36]:
by(iris, :Species, df -> (mean(df[:PetalWidth]), mean(df[:PetalLength])))

,Species,x1
1,setosa,"(0.24600000000000002,1.462)"
2,versicolor,"(1.3259999999999998,4.260000000000001)"
3,virginica,"(2.026,5.5520000000000005)"
